# Import Libraries

In [ ]:
#System Libraries
from datetime import datetime
from pandas.tseries.offsets import BDay

#DataFrames manipulation
import pandas as pd

#Data download from Yahoo Finance
from yahooquery import Ticker
from fake_useragent import UserAgent

#Download Tables from websites
import requests

#Track the progress of the loop
from tqdm import tqdm 

#Libraries for the Plotting
import holoviews as hv
from holoviews import opts, dim
from holoviews.plotting.links import RangeToolLink
from bokeh.models import HoverTool

#Librarie to save the plots to html object
import panel as pn
import param

hv.extension('bokeh')

# New York Fed - Download the data and Create the Chart

In [ ]:
#Download the File with the data from New York Fed website
ny_fed_data = pd.read_excel('https://www.newyorkfed.org/medialibrary/media/research/capital_markets/allmonth.xls')

In [ ]:
#Cleaning the Dataframe to get only the data we need and to get it in the format to convert to the graph
recession_probability_df = ny_fed_data[['Date','Rec_prob']]
recession_probability_df = recession_probability_df.dropna()
recession_probability_df = recession_probability_df.rename(columns = {'Rec_prob': 'Probability'})
recession_probability_df['Probability'] = recession_probability_df['Probability'] * 100
recession_probability_df['Date'] = recession_probability_df['Date'].dt.date
recession_probability_df = recession_probability_df.set_index('Date')

In [ ]:
#Instantiate the Dataframe for the "Probability of US Recession Predicted by Treasury Spread - Twelve Months Ahead" 
graph_df = recession_probability_df

#Generate all curves
def getCurves(n):
    for column in graph_df.columns:
        hover = HoverTool(tooltips=[("Date", "@Date{%F}"), (column, f"@{column}")], formatters={'@Date': 'datetime'})  
        curve = hv.Curve(graph_df[column], label = column).opts(opts.Curve(tools=[hover]))
        curve = curve.opts(xticks=10)
        yield curve
        
source_curves, target_curves  = [], []
for curve in getCurves(2):
    
    src = curve.relabel('').opts(width=800, height=100, yaxis=None) 
    tgt = curve.opts(width=800, ylabel = 'Probability of US Recession (%)')
    source_curves.append(src)
    target_curves.append(tgt)
    
# Link RangeTool for the first curves in the list.
RangeToolLink(source_curves[0],target_curves[0], axes=['x','y'])  

#Overlay the source and target curves 
overlaid_plot_src = hv.Overlay(source_curves).relabel('')
overlaid_plot_tgt = hv.Overlay(target_curves)

#Add an Horizontal Line at the last value
last_value = recession_probability_df['Probability'].iloc[-1]
hline = hv.HLine(last_value).opts(color='red', line_width=1, line_dash='5 5')
overlaid_plot_tgt = overlaid_plot_tgt * hline
overlaid_plot_tgt = overlaid_plot_tgt.relabel('Probability of US Recession Predicted by Treasury Spread - Twelve Months Ahead').opts(
    height=400, legend_position='top')

# Layout the plot
full_graph_initial = (overlaid_plot_tgt + overlaid_plot_src).cols(1)

In [ ]:
#Save the Plots
p = pn.panel(full_graph_initial)
p.save('Bankrupcy_Article_Recession_Probability.html', embed = True)

# Russell 3000 Index - Download the Financial Data and Calculate the Z-Score

In [ ]:
#Import the Russell 3000 Index (RAY) based on IWV ETF
ray_df = pd.read_csv('IWV_holdings_12302022_clean.csv')

#Filter for the Stocks - Delete Cash, Futures and so on
ray_df = ray_df[ray_df['Asset Class'] == 'Equity']

#Get the Ticker list
ticker_list = ray_df['Ticker'].to_list()

In [ ]:
#Create an empty DataFrame where we will Concatenate the information
final_df_full = pd.DataFrame(columns=['Ticker', 'Name', 'Altman Z-Score','Probability of Default'])

In [ ]:
for ticker in tqdm(ticker_list):
    try:
        #Instantiate the Yahoo Finance and UserAgent module
        ua = UserAgent()
        stock = Ticker(ticker, user_agent=ua.random)

        #Download the Balance Sheet and Income Statement from Yahoo Finance
        balance_sheet_df = stock.balance_sheet(frequency='q')
        income_statement_df = stock.income_statement(frequency='q')

        #Delete data for 2023-03-31 in case the company already presented earnings
        balance_sheet_df = balance_sheet_df.iloc[:-1] if balance_sheet_df.iloc[-1][0].strftime('%Y-%m-%d') == '2023-03-31' else balance_sheet_df
        income_statement_df.iloc[:-2] if income_statement_df.iloc[-1][0].strftime('%Y-%m-%d') == '2023-03-31' else income_statement_df

        #Limit the Data for the Last Period of the Balance Sheet and Income Statement  
        balance_sheet_df = pd.DataFrame(balance_sheet_df.iloc[-1]).T
        income_statement_df = pd.DataFrame(income_statement_df.iloc[-1]).T

        #Obtain the Datapoints needed for the Ratios calculations
        total_assets = balance_sheet_df['TotalAssets'][0]
        working_capital = 0 if balance_sheet_df.get('WorkingCapital') is None else balance_sheet_df['WorkingCapital'][0]
        retained_earnings = balance_sheet_df['RetainedEarnings'][0]
        total_liabilities = balance_sheet_df['TotalLiabilitiesNetMinorityInterest'][0]
        ebit = income_statement_df['PretaxIncome'][0] if income_statement_df.get('EBIT') is None else income_statement_df['EBIT'][0]
        sales = income_statement_df['TotalRevenue'][0]

        #Get the price for the end of the quarter under analysis
        date = balance_sheet_df['asOfDate'][0].date()
        price_df = stock.history(start = (date - BDay(1)).strftime('%Y-%m-%d'), end= date.strftime('%Y-%m-%d'))
        price_df = price_df.reset_index()
        price = price_df['close'][0]

        #Get the Maket Value of Equity
        shares_outstanding = balance_sheet_df['OrdinarySharesNumber'][0] 
        mv_equity = price * shares_outstanding

        #Calculate the ratios
        working_capital_to_assets = working_capital/total_assets
        ret_earnings_to_assets = retained_earnings/total_assets
        ebit_to_assets = ebit/total_assets
        mv_equity_to_bv_liabilities = mv_equity/total_liabilities
        sales_to_assets = sales/total_assets

        #Calculate the Altman Z score
        altman_zscore = round((1.2 * working_capital_to_assets + 1.4 * ret_earnings_to_assets + 3.3 * ebit_to_assets +
                        0.6 * mv_equity_to_bv_liabilities + 1.0 * sales_to_assets), 2)

        #Get the Key to See if theProbability of Default is Low, High or Uncertain
        if altman_zscore > 3.0:
            bankruptcy_key = 'Low'
        elif altman_zscore <= 3.0 and altman_zscore >= 1.81:
            bankruptcy_key = 'Uncertain'
        else:
            bankruptcy_key ='High'

        #Create a DataFrame with the Final Data for each Stock
        name = ray_df[ray_df['Ticker'] == ticker]['Name'].iloc[0]
        final_df = pd.DataFrame([{'Ticker':ticker, 'Name':name, 'Altman Z-Score':altman_zscore, 'Probability of Default':bankruptcy_key}])

        #Concatenate current Ticker's Data to the Full DataFrame
        final_df_full = pd.concat([final_df_full, final_df], axis=0)
        
    except:
        next
        
#Add a Column with Sector
final_df_full = final_df_full.merge(ray_df[['Ticker','Sector']], on='Ticker', how='left')

#Reorder column order
final_df_full = final_df_full[['Ticker', 'Name', 'Sector', 'Altman Z-Score', 'Probability of Default']]

#Remove the NaNs
final_df_full = final_df_full.dropna()

In [ ]:
#Create Holoviews table object for All Companies
full_df_table = hv.Table(final_df_full).opts(
    opts.Table(width=950, height=280, selectable = True, index_position = None, 
               title = 'Altman Z-Score for All Companies'))

#Save the Plots
p = pn.panel(full_df_table)
p.save('Altman_ZScore_All_Companies.html', embed = True)

# Create the Table with the Top 10 Largest Companies

In [ ]:
#Create a Dataframe with Only the Top 10
top_10_df = final_df_full[0:10]

In [ ]:
#Create Holoviews table object for the Top 10 Table
top_10_table = hv.Table(top_10_df).opts(
    opts.Table(width=950, height=280, selectable = True, index_position = None, 
               title = 'Altman Z-Score for Top 10 Largest Companies by Market Cap'))

#Save the Plots
p = pn.panel(top_10_table)
p.save('Altman_ZScore_Top10_Companies.html', embed = True)

# Data for the List of Companies which filled for Bankruptcy so far in 2023

In [ ]:
#Get the table with the data for the Bankruptcies that already occured in 2023 from StockAnalysis website
ua = UserAgent()
header = {"User-Agent": ua.random}

r = requests.get('https://stockanalysis.com/actions/bankruptcies/2023/', headers = header)

bankrupcies_table = pd.read_html(r.content)[0]

bankrupcies_ticker_list = bankrupcies_table['Symbol'].to_list()

In [ ]:
#Create an empty DataFrame where we will Concatenate the information
bankruptcies_df_full = pd.DataFrame(columns=['Ticker', 'Name', 'Altman Z-Score','Probability of Default'])

#Run the Loop
for ticker in tqdm(bankrupcies_ticker_list):
    try:
        
        #Instantiate the Yahoo Finance and UserAgent module
        ua = UserAgent()
        stock = Ticker(ticker, user_agent=ua.random)

        #Download the Balance Sheet and Income Statement from Yahoo Finance
        balance_sheet_df = stock.balance_sheet(frequency='q')
        income_statement_df = stock.income_statement(frequency='q')

        #Delete data for 2023-03-31 in case the company already presented earnings
        balance_sheet_df = balance_sheet_df.iloc[:-1] if balance_sheet_df.iloc[-1][0].strftime('%Y-%m-%d') == '2023-03-31' else balance_sheet_df
        income_statement_df.iloc[:-2] if income_statement_df.iloc[-1][0].strftime('%Y-%m-%d') == '2023-03-31' else income_statement_df

        #Limit the Data for the Last Period of the Balance Sheet Statement  
        balance_sheet_df = pd.DataFrame(balance_sheet_df.iloc[-1]).T
        income_statement_df = pd.DataFrame(income_statement_df.iloc[-1]).T

        #Obtain the Datapoints needed for the Ratios calculations
        total_assets = balance_sheet_df['TotalAssets'][0]
        working_capital = 0 if balance_sheet_df.get('WorkingCapital') is None else balance_sheet_df['WorkingCapital'][0]
        retained_earnings = balance_sheet_df['RetainedEarnings'][0]
        total_liabilities = balance_sheet_df['TotalLiabilitiesNetMinorityInterest'][0]
        ebit = income_statement_df['PretaxIncome'][0] if income_statement_df.get('EBIT') is None else income_statement_df['EBIT'][0]
        sales = income_statement_df['TotalRevenue'][0]

        #Get the price for the end of the quarter under analysis
        date = balance_sheet_df['asOfDate'][0].date()
        price_df = stock.history(start = (date - BDay(1)).strftime('%Y-%m-%d'), end= date.strftime('%Y-%m-%d'))
        price_df = price_df.reset_index()
        price = price_df['close'][0]

        #Get the Maket Value of Equity
        shares_outstanding = balance_sheet_df['OrdinarySharesNumber'][0] 
        mv_equity = price * shares_outstanding

        #Calculate the ratios
        working_capital_to_assets = working_capital/total_assets
        ret_earnings_to_assets = retained_earnings/total_assets
        ebit_to_assets = ebit/total_assets
        mv_equity_to_bv_liabilities = mv_equity/total_liabilities
        sales_to_assets = sales/total_assets

        #Calculate the Altman Z score
        altman_zscore = round((1.2 * working_capital_to_assets + 1.4 * ret_earnings_to_assets + 3.3 * ebit_to_assets +
                        0.6 * mv_equity_to_bv_liabilities + 1.0 * sales_to_assets), 2)

        #Get the Key to See if theProbability of Default is Low, High or Uncertain
        if altman_zscore > 3.0:
            bankruptcy_key = 'Low'
        elif altman_zscore <= 3.0 and altman_zscore >= 1.81:
            bankruptcy_key = 'Uncertain'
        else:
            bankruptcy_key ='High'

        #Create a DataFrame with the Final Data for each Stock
        name = bankrupcies_table[bankrupcies_table['Symbol'] == ticker]['Company Name'].iloc[0]
        final_df = pd.DataFrame([{'Ticker':ticker, 'Name':name, 'Altman Z-Score':altman_zscore, 'Probability of Default':bankruptcy_key}])

        #Concatenate current Ticker's Data to the Full DataFrame
        bankruptcies_df_full = pd.concat([bankruptcies_df_full, final_df], axis=0)
        
    except:
        next
                
#Remove the NaNs
bankruptcies_df_full = bankruptcies_df_full.dropna()

In [ ]:
#Create Holoviews table object for the Bankruptices Table
bankruptcies_table = hv.Table(bankruptcies_df_full).opts(
    opts.Table(width=950, height=280, selectable = True, index_position = None, 
               title = 'Altman Z-Score for Companies which Filled for Bankruptcy in 2023'))

#Save the Plots
p = pn.panel(bankruptcies_table)
p.save('Altman_ZScore_Bankrupt_Companies.html', embed = True)

# Create the Table with the Companies with High Probability of Default

In [ ]:
#Create a Dataframe with Only the Companies with High Probability of Default
high_prob_default_df = final_df_full[final_df_full['Probability of Default'] == 'High']

In [ ]:
#Create Holoviews table object for the Top 10 Table
high_prob_default_table = hv.Table(high_prob_default_df).opts(
    opts.Table(width=950, height=280, selectable = True, index_position = None, 
               title = 'Altman Z-Score for the Companies with High Probability of Default'))

#Save the Plots
p = pn.panel(high_prob_default_table)
p.save('Altman_ZScore_High_Prob_Default_Companies.html', embed = True)